## Kernel: Python 3.8

In [ ]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [ ]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        }
        account = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=account, headers=headers)
        token = response.json()['data'].get('access_token')
    
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    
    return token, account

(token, account) = login()
headers = dict()
headers["Accept"] = 'application/json'
headers["Authorization"] = f'Bearer {token}'
headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

print(f'login to {account["email"]} success')

## 1. Fetch ALL Mitra (Raw & Test)

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=20&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
reports = response.json().get('data')

print(reports)

## 2. Process Data FULL

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

id_cols = []
name_cols = []
activity_name_cols = []
quotas_cols = []

for mitra_data_detail in mitra_data:
    # fetch data from mitra_data_detail dict in mitra_data list
    id = mitra_data_detail["id"]
    name = mitra_data_detail["name"]
    activity_name = mitra_data_detail["activity_name"]
    quotas = mitra_data_detail["total"]

    # Append Data
    id_cols.append(id)
    name_cols.append(name)
    activity_name_cols.append(activity_name)
    quotas_cols.append(quotas)

mitra_data_df_full = pd.DataFrame({
    "id": id_cols,
    "name": name_cols,
    "activity_name": activity_name_cols,
    "quotas": quotas_cols
})


## 2. Process Data COMPACT

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

id_cols = []
name_cols = []
activity_name_cols = []
quotas_cols = []

for mitra_data_detail in mitra_data:
    # fetch data from mitra_data_detail dict in mitra_data list
    id = mitra_data_detail["id"]
    name = mitra_data_detail["name"]
    activity_name = mitra_data_detail["activity_name"]
    quotas = mitra_data_detail["total"]

    # Append Data
    id_cols.append(id)
    name_cols.append(name)
    activity_name_cols.append(activity_name)
    quotas_cols.append(quotas)

mitra_data_df_compact = pd.DataFrame({
    "id": id_cols,
    "name": name_cols,
    "activity_name": activity_name_cols,
    "quotas": quotas_cols
})


In [ ]:
mitra_data_df_full.info()

In [ ]:
mitra_data_df_compact.info()

In [ ]:
mitra_data_df_full

## 3. Output Files [CSV and EXCEL]

In [ ]:
def output_files(df: pd.DataFrame):
    output_types = ["CSV", "Excel"]
    
    print("=== Output File Types ===")
    
    for i in range(len(output_types)):
        format_name = output_types[i]
        print(f'{i+1}. {format_name}')
    
    choose_number = int(input("Pilih format file untuk mengekspor dataframe: ")) - 1

    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")

    if choose_number == 0:
        filename = './' + dt_string + ' - data mitra mbkm.csv'
        df.to_csv(filename, index=False)
    elif choose_number == 1:
        filename = './' + dt_string + ' - data mitra mbkm.xlsx'
        df.to_excel(filename, index=False)
    else:
        raise ValueError("Not a valid input")

    print(f"Creating {output_types[choose_number]} with a file name suffix `- data mitra mbkm`")

output_files(mitra_data_df_full)

## 4. Output Files [JSON]

In [ ]:
def output_files_json(df: pd.DataFrame):
    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")
    filename = './' + dt_string + ' - data mitra mbkm.json'
    df.to_json(filename, orient="records")

output_files_json(mitra_data_df_full)

## Extra: 1. Time Config

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

print("now =", now)

# dd-mm-YY H.M
dt_string = now.strftime("%d-%m-%Y %H.%M")
filename ='./' + dt_string + ' - data mitra mbkm.csv'
print("date and time =", filename)